# Segmenting and Clustering Neighborhoods in Toronto

## Applied Data Science Capstone, Week 3

## Peer-Graded Assignment

### 1. Creating the Toronto dataframe

#### Let's start by downloading all of our dependencies

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

We're going to need <b>BeautifulSoup 4</b> for scraping the info from Wikipedia. Let's load it...

In [3]:
from bs4 import BeautifulSoup

In [46]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

postal_page=requests.get(url)

postal_page.content

soup = BeautifulSoup(postal_page.content, 'html.parser')

#### Now we can finally create our Toronto dataframe

In [80]:
#using pandas to read the table into a dataframe
toronto_table = soup.find('table')
df = pd.read_html(str(toronto_table))

toronto_df = df[0]
toronto_df.rename(columns={"Postcode": "PostalCode", "Neighbourhood": "Neighborhood"}, inplace = True)
toronto_df.head()

#another approach, which relies more on BeautifulSoup, to create the postal code df is below
    #toronto_table = []
    #table_rows = toronto_table_elem.find_all('tr')
        #for tr in table_rows:
            #td = tr.find_all('td')
            #row = [tr.text for tr in td]
            #toronto_table.append(row)
    #toronto_df = pd.DataFrame(toronto_table, columns=["PostalCode", "Borough", "Neighborhood"])

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Ok, looking good so far! 
Now we need to <b>drop</b> the cells where <b>Borough = Not assigned</b>, and we need to <i>update</i> any rows where Neighborhood = Not Assigned so that <i>Neighborhood = Borough</i>.

In [83]:
toronto_updated = toronto_df[toronto_df.Borough != 'Not assigned']
toronto_updated.loc[toronto_updated['Neighborhood'] == 'Not assigned', ['Neighborhood']] = toronto_updated['Borough']
toronto_updated

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


I saved the best for last... maybe? Not really, this was just the most challenging part of the problem for me. <br>
Now it's time to tackle the following: <br>
<i>More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.</i>

In [94]:
toronto_final = toronto_updated.groupby('PostalCode', as_index=False).agg(lambda x:', '.join(set(x)))

toronto_final

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Port Union, Highland Creek, Rouge Hill"
2,M1E,Scarborough,"West Hill, Guildwood, Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, East Birchmount Park, Ionview"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


#### The final part of this section is to print the shape of this dataframe, which is below

In [95]:
toronto_final.shape

(103, 3)